In [1]:
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append('../AxiScan/')
import dataGen
import LL_Scan
import pymultinest

In [6]:
########################
###    Seed Values   ###
########################
c = 299798.452

# Physics Parameters
ma = 5.5e5*2*np.pi
A = 8000.0
PSDback= 163539.36

# Bulk SHM Parameters
v0_Halo = 220.0
vDotMag_Halo = 232.36
alpha_Halo = .49
tbar_Halo = 72.40

# Sagitarrius Stream Parameters
v0_Stream = 10.0
vDotMag_Stream = 418.815
alpha_Stream = .65903
tbar_Stream = 279.51
fracStream = 0.0 #0.05

# Data Output Size
freqs = np.linspace(.99998, 1.00002, 10000)*5.5e5
PSD_Data = np.zeros((365, len(freqs)))

In [3]:
# Instantiate the data generator
generator = dataGen.Generator(ma, A, PSDback, v0_Halo, vDotMag_Halo, alpha_Halo, tbar_Halo,
                             v0_Stream, vDotMag_Stream, alpha_Stream, tbar_Stream, fracStream, freqs)

# Fill the PSD_Data array
for i in range(365):
    PSD_Data[i] = np.array(generator.makePSDFast(i))

# Average over the days in the PSD_Data array for the coarse scan
Stacked_PSD_Data = np.mean(PSD_Data, axis = 0)

In [4]:
# Construct the test set of masses
N_testMass = int(np.log(freqs[-1] / freqs[0])  / np.log(1. + v0_Halo**2. / 2. / c**2.))
mass_TestSet = freqs[0]*(1. + v0_Halo**2. / 2. / c**2.)**np.arange(N_testMass) * 2*np.pi

# Construct the test set of A_Vals
A_TestSet = np.linspace(.5, 1.5, 20)*A

In [5]:
# A_Vals to test in Multinest Scan
theta_min = [0.0]
theta_max = [20000.0]

theta_interval = list(np.array(theta_max) - np.array(theta_min))
n_params = len(theta_interval)

def prior_cube(cube, ndim=1, nparams=1):
    """ Cube of priors - in the format required by MultiNest
    """

    for i in range(ndim):
        cube[i] = cube[i] * theta_interval[i] + theta_min[i]
    return cube